In [3]:
from pathlib import Path
"Module" in str(Path.cwd())

True

In [4]:
from XQuant import Tools
Tools.get_newest_file("ResConSecTarpriScore")

'E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5'

In [5]:
import h5py

In [40]:
data = h5py.File('E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5', 'r')

In [15]:
data["a"].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values', 'block2_items', 'block2_values']>

In [67]:
"S" in data["a"]["axis0"].dtype

TypeError: argument of type 'numpy.dtype[bytes_]' is not iterable

In [54]:
data["a"]["axis0"]

<HDF5 dataset "axis0": shape (12,), type "|S16">

In [63]:
data["a"]["axis1"].dtype == "int64"

True

In [49]:
data["a"]["axis0"][:][0].decode("utf-8") == "secCode"

AttributeError: 'numpy.ndarray' object has no attribute 'decode'

In [38]:
import pandas as pd
data = pd.HDFStore('E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5', mode='r')

In [39]:
data["a"]

,secCode,secName,repForeDate,conTarPriceType,conTarPrice,tarPriceLastDate,conScoreType,conScore,scoreLastDate,closePrice,conTarReturn,updateTime
0,000400,许继电气,2023-01-03,2,24.25,2022-10-31,1,1.00,2022-10-31,20.71,0.1709,2023-01-03 17:36:27
1,000400,许继电气,2023-01-04,1,24.15,2023-01-03,1,1.00,2023-01-03,20.40,0.1838,2023-01-04 17:36:13
2,000400,许继电气,2023-01-05,1,24.15,2023-01-03,1,1.00,2023-01-03,20.84,0.1588,2023-01-05 17:36:19
3,000400,许继电气,2023-01-06,1,24.15,2023-01-03,1,1.00,2023-01-03,21.37,0.1301,2023-01-06 17:36:24
4,000400,许继电气,2023-01-09,1,24.15,2023-01-03,1,1.00,2023-01-03,21.71,0.1124,2023-01-09 17:36:11
...,...,...,...,...,...,...,...,...,...,...,...,...
137,002683,广东宏大,2023-07-28,3,NaN,NaN,2,0.75,2023-05-08,26.79,NaN,2023-07-28 17:32:17
137,002164,宁波东力,2023-07-28,3,NaN,NaN,3,NaN,NaN,5.87,NaN,2023-07-28 17:32:17
137,600165,宁科生物,2023-07-28,3,NaN,NaN,3,NaN,NaN,4.37,NaN,2023-07-28 17:32:18
110,603307,扬州金泉,2023-07-28,3,NaN,NaN,2,1.00,2023-05-18,49.43,NaN,2023-07-28 17:32:19


In [36]:
data["a"]["block1_items"][:]

array([b'conTarPriceType', b'conScoreType'], dtype='|S15')

In [43]:
data["a"]["block0_items"][:]

array([b'conTarPrice', b'conScore', b'closePrice', b'conTarReturn'],
      dtype='|S12')

In [35]:
data["a"]["block2_items"][:]

array([b'secCode', b'secName', b'repForeDate', b'tarPriceLastDate',
       b'scoreLastDate', b'updateTime'], dtype='|S16')